In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd
import os
import re
from functions import allMondays, intHolidayClosures, create_trips, country_holidays

In [5]:
#constants
URL = 'https://camper.fti.de/'
DELAY = 3
PAGE_LOAD_ID = "SearchMask"
COOKIE_ID = '//*[@id="uc-center-container"]/div[2]/div/div[1]/div/button'
COOKIE_JS_PATH = 'document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-gWXbKe.fSIThp > div > div.sc-cxpSdN.ihpUDX > div > button")'
NOTIFICATION_JS_PATH = 'document.querySelector("#pid-14572 > div.cleverpush-confirm.cleverpush-confirm-modal.cleverpush-confirm-windows.cleverpush-confirm-chrome.cleverpush-confirm-chrome.cleverpush-confirm-1 > div.cleverpush-confirm-wrap > div.cleverpush-clearfix.cleverpush-clearfix-bottom > div.cleverpush-confirm-buttons > button.cleverpush-confirm-btn.cleverpush-confirm-btn-deny")'
POPUP_JS_PATH = 'document.querySelector("#trboModule_12920_202268_88_container > div.trbo-closeButton.trbo-theme-0.trboType-icon > i")'
CITY = 'Los Angeles'
CITY_JS_PATH = 'document.querySelector("#SearchMask > div.flexContainer.location > div:nth-child(1) > div:nth-child(1) > div > div > input")'
CAL_ELEMS_XPATH = "//span[contains(@class, 'cell day')]"
PU_CALENDAR_JS_PATH = 'document.querySelector("#SearchMask > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(1) > input")'
DO_CALENDAR_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(1) > input')"
DO_MONTH_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > header > span.day__month_btn.up')"
PU_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
PU_MONTH_NEXT_BTN_CLASS_NAME = "span[class = 'next']"
DO_MONTH_TEXT_XPATH = "//*[@id='SearchMask']/div[2]/div[2]/div/div/div[2]/header/span[3]"
SUBMIT_BUTTON_ID = "SubmitButton"
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"
# PICKUP_LOC_ID = 'pickUpLocation'
# PICKUP_LOC_XPATH = "//select[@id = 'pickUpLocation]"

In [18]:
def get_proxies():

    options = webdriver.ChromeOptions()
    #maximize argument
    options.add_argument("start-maximized")

    #removes "chrome is being controlled by automated software"
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, chrome_options= options)
    driver.get("https://sslproxies.org/")

    #scroll to the table
    driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered']//th[contains(., 'IP Address')]"))))

    ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 1]")))]
    ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 2]")))]

    proxies = []
    for i in range(0, len(ips)):
        proxies.append(ips[i]+':'+ports[i])
    #print(proxies)
    
    driver.quit()

    return proxies

proxies = get_proxies()


proxies

C:\Users\mriedeman\AppData\Local\Temp\ipykernel_10356\3865060908.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options= options)


['71.86.129.131:8080',
 '198.11.175.192:8080',
 '49.0.2.242:8090',
 '20.54.56.26:8080',
 '47.252.1.180:3128',
 '185.125.125.157:80',
 '83.229.73.175:80',
 '130.41.55.190:8080',
 '134.84.66.194:3128',
 '3.14.253.82:80',
 '41.65.174.120:1981',
 '5.202.95.147:8080',
 '110.238.109.146:8080',
 '201.229.250.22:8080',
 '172.104.241.29:8081',
 '51.159.207.156:3128',
 '110.238.113.119:8080',
 '147.139.133.146:8080',
 '149.129.239.170:8080',
 '161.97.126.37:8118',
 '35.187.4.12:8888',
 '47.245.34.161:8080',
 '172.104.252.86:80',
 '45.33.12.251:8080',
 '178.79.191.47:54417',
 '45.79.27.210:44554',
 '149.129.184.250:8080',
 '172.105.196.53:9000',
 '154.236.177.100:1981',
 '95.79.25.77:3128',
 '213.222.34.200:53281',
 '103.145.253.237:3128',
 '96.126.124.197:81',
 '47.254.195.196:8080',
 '176.56.107.184:52210',
 '114.116.2.116:8001',
 '178.79.138.253:8080',
 '130.185.119.20:3128',
 '46.229.214.206:3128',
 '173.212.200.30:3128',
 '189.173.168.174:999',
 '157.100.12.138:999',
 '43.204.1.146:80',
 '16

In [19]:
def create_browser(URL, ELEMENT_ID):

    """Creates the browser and loads the page given url and webelement"""
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'--proxy-server={proxies[0]}')
    s=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=s)

    browser.get(URL)

    try:
        myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, ELEMENT_ID)))
        print ("Page is ready!")
    except TimeoutException:
        print ("Couldn't find element")

    return browser


browser = create_browser(URL, PAGE_LOAD_ID)

Page is ready!


In [20]:
#accept cookies

#not sure if this will work
WebDriverWait(browser, 5, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for Accept Cookies alert to appear')



browser.execute_script(f"return {COOKIE_JS_PATH}").click()

TimeoutException: Message: Timed out waiting for Accept Cookies alert to appear


In [7]:
#deny news

#not sure if this will work
WebDriverWait(browser, 15, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for News alert to appear')
browser.execute_script(f"return {NOTIFICATION_JS_PATH}").click()

In [ ]:
#close popup
WebDriverWait(browser, 15, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for News alert to appear')
browser.execute_script(f"return {POPUP_JS_PATH}").click()

In [6]:
#use if running code below more than once

browser.execute_script(f"return {CITY_JS_PATH}").click()
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").clear()

In [7]:
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(f'{CITY}')
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(Keys.RETURN)

In [8]:
#simulate a single trip
german_months = {
    1 : 'Jan',
    2 : 'Feb',
    3 : 'Mär',
    4 : 'Apr',
    5 : 'Mai',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Aug',
    9 : 'Sep',
    10: 'Okt',
    11: 'Nov',
    12: 'Dez'

}

start_date = date(2022, 11, 14)
end_date = start_date + timedelta(17)
trip = {}
trip['start_date'] = start_date
trip['start_month'] = german_months[start_date.month]
trip['start_day'] = start_date.day
trip['end_date'] = end_date
trip['end_day'] = end_date.day
trip['end_month'] = german_months[end_date.month]
trip['end_day']
trip['city'] = CITY

In [9]:
#open the pickup month calendar
browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()

In [10]:
#select the pickup month
cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]

while cal_month != trip['start_month']:
    
    browser.find_element(By.CSS_SELECTOR, f"{PU_MONTH_NEXT_BTN_CLASS_NAME }").click()
    time.sleep(1)
    cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]
    print(cal_month)
    

In [11]:
#click on the pickup day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['start_day'])].click()

In [12]:
#open the drop off calendar

browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()

In [37]:
#select the drop off month
cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
while cal_month != trip['end_month']:
    browser.find_element(By.XPATH, f"{DO_MONTH_TEXT_XPATH}").click()
    #updates cal_month variable to whatever date you are currently on
    time.sleep(int(np.random.rand(1)*3))
    cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
    print(cal_month)

Dez


In [13]:
#select drop off day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['end_day'])].click()

In [14]:
#click search
browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()

0


In [ ]:
#explicity wait for load
for j in range(1)   
    i = 0 
    LOADING_ICON = "MainLoader"
    while i < 5:
        try:
            myElem = WebDriverWait(browser, DELAY).until_not(EC.presence_of_element_located((By.ID, LOADING_ICON)))
            print ("Page is ready!")
        except TimeoutException:
            print ("still Loading..")
            i+=1
            print("Trying again...")

            if i == 3:
                browser.refresh()
                print("Refresshing page to try again. Connection May be slow")
                time.sleep(3)
        
    if i == 5:
        print("Could not Load page after several attempts. Refreshing the browser and beginning with the next trip.")
        browser.get(URL)
        continue

    

In [40]:
#ADD LOGIC TO WAIT FOR PAGE TO LOAD AND GO BACK IF THERE ARE NO RESULTS

In [15]:
#load all the results
browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()

AttributeError: 'NoneType' object has no attribute 'click'

In [16]:
LISTINGS_CLASS_NAME = "resultCard"
listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")
listings

[<selenium.webdriver.remote.webelement.WebElement (session="2eccb4e978c4e2ce8ae94070023b2d14", element="8fba363d-d05b-4869-a8e1-40636b57a894")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2eccb4e978c4e2ce8ae94070023b2d14", element="4d219ee3-022f-4eb6-8667-b09f18bf4738")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2eccb4e978c4e2ce8ae94070023b2d14", element="6474e87a-852b-4d60-90a7-127e615e1b69")>]

In [51]:
#listings[0].find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]

'Station Wagon'

In [57]:
#listings[0].find_element(By.CLASS_NAME, "camper-details").text

'auf Anfrage\nUnbegrenzte Freimeilen\nFahrzeugversicherung mit Selbstbeteiligung USD 3.000\nZusatzversicherung zur Rückerstattung der Selbstbeteiligung\nEUR 10 Mio. Zusatz-Haftpflichtversicherung\nZusatzfahrer\nGesamtpreis\n1171,00 EUR\n68,88 EUR pro Tag'

In [65]:
#money = listings[0].find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')

In [69]:
# import re

# '.'.join(re.findall(r'\d+', money[0]))

'1171.00'

In [17]:
trip_data = []
for card in listings:

    vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
    company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
    price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
    total_price = '.'.join(re.findall(r'\d+', price_string[0]))
    daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


    trip_data.append({
    'city': trip['city'],
    'start_date':trip['start_date'],
    'end_date':trip['end_date'],
    'total_price':total_price,
    'daily_price': daily_price,
    'vehicle_class':vehicle_class,
    'company_name':company_name
    })

pd.DataFrame(trip_data)

IndexError: list index out of range

In [79]:
trip_data_df = pd.DataFrame(trip_data)

In [81]:
URL

'https://camper.fti.de/'

In [82]:
try: 
    os.makedirs(f"../Output Data/FTI Camper/{trip['city']}")
except FileExistsError:
    pass

trip_data_df.to_csv(f"../Output Data/FTI Camper/{trip['city']}/{trip['city']} {date.today()}.csv", index=False)